In [ ]:
# Simple example that uses prepared draw function
import webgpu.jupyter
from ngsolve import *

m = Mesh(unit_square.GenerateMesh(maxh=0.1))

webgpu.jupyter.Draw(x, m)

In [ ]:

import webgpu.jupyter
#from webgpu.mesh import MeshData
from ngsolve import *

def render(data):
    import ngsolve_webgpu.mesh
    import pyodide.ffi
    import webgpu
    from webgpu.jupyter import gpu
    print("render data", data)
    import ngsolve as ngs
    import js

    mesh = data["mesh"]
    cf = data["cf"]
    order = data.get("order", 1)

    mesh_data = ngsolve_webgpu.mesh.MeshData(mesh, cf, order)
    mesh_object = ngsolve_webgpu.mesh.MeshRenderObject(gpu, mesh_data)
    gpu.u_function.min = 0
    gpu.u_function.max = 1.0
    def render_function(t):
        gpu.update_uniforms()
        # gpu.uniforms.update_buffer()

        encoder = gpu.device.createCommandEncoder()
        mesh_object.render(encoder)
        gpu.device.queue.submit([encoder.finish()])

    render_function = pyodide.ffi.create_proxy(render_function)
    gpu.input_handler.render_function = render_function
    js.requestAnimationFrame(render_function)

m = Mesh(unit_square.GenerateMesh(maxh=.1))
cf=sin(10*x)*sin(10*y)

# "data" will be passed as argument to render on the pyodide side
webgpu.jupyter.DrawCustom({"cf": cf, "mesh": m, "order": 5}, render, ["ngsolve_webgpu"])